In [121]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import copy, cv2,glob, shutil
import os
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import zipfile

In [122]:
class mC(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.96):
            self.model.stop_training = True
callbacks = mC()

In [123]:
from google.colab import drive

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip

--2022-07-30 09:23:36--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.123.128, 142.250.98.128, 142.250.97.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.123.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M   194MB/s    in 1.0s    

2022-07-30 09:23:37 (194 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]



In [131]:
lz = "/tmp/rps.zip"
zip_ref = zipfile.ZipFile(lz, 'r')
zip_ref.extractall('/tmp/data')
zip_ref.close()

In [144]:
base_dir = '/tmp/rps'
data_dir = '/tmp/data/rps'
t_dir = os.path.join(base_dir,'train')
val_dir = os.path.join(base_dir,'val')
roc_dir = os.path.join(data_dir,'rock')
pa_dir = os.path.join(data_dir,'paper')
sci_dir = os.path.join(data_dir,'scissors')

In [133]:
os.mkdir(base_dir)
os.mkdir(t_dir)
os.mkdir(val_dir)

FileExistsError: ignored

In [145]:
tr = os.path.join(t_dir, 'rock')
tp = os.path.join(t_dir, 'paper')
ts = os.path.join(t_dir, 'scissors')
vr = os.path.join(val_dir, 'rock')
vp = os.path.join(val_dir, 'paper')
vs = os.path.join(val_dir, 'scissors')

In [146]:
os.mkdir(tr)
os.mkdir(tp)
os.mkdir(ts)
os.mkdir(vr)
os.mkdir(vp)
os.mkdir(vs)

FileExistsError: ignored

In [147]:
train_sci_dir, val_sci_dir = train_test_split(os.listdir(sci_dir), test_size = 0.30)
train_roc_dir, val_roc_dir = train_test_split(os.listdir(roc_dir), test_size = 0.30)
train_pa_dir, val_pa_dir = train_test_split(os.listdir(pa_dir), test_size = 0.30)

In [142]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.35,
    fill_mode = 'nearest',
)

test_datagen = ImageDataGenerator(
    rescale = 1./225,
    rotation_range = 20,
    horizontal_flip = True,
    vertical_flip = True,
    shear_range = 0.35,
    fill_mode = 'nearest'
)

In [155]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150,150),
    batch_size= 32,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 2520 images belonging to 3 classes.
Found 2520 images belonging to 3 classes.


In [156]:
#Buatlah model dengan output layer 3
model = tf.keras.models.Sequential([
  tf.keras.layers.Convolution2D(
      input_shape=(150,150,3),
      filters=64,
      kernel_size=3,
      activation=tf.keras.activations.relu
  ),
  tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),
    strides=(2, 2)
  ),
  tf.keras.layers.Convolution2D(
      filters=64,
      kernel_size=3,
      activation=tf.keras.activations.relu
  ),
  tf.keras.layers.MaxPooling2D(
      pool_size=(2, 2),
      strides=(2, 2)
  ),
  tf.keras.layers.Convolution2D(
      filters=128,
      kernel_size=3,
      activation=tf.keras.activations.relu
  ),
  tf.keras.layers.MaxPooling2D(
      pool_size=(2, 2),
      strides=(2, 2)
  ),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(
      units=512,
      activation=tf.keras.activations.relu
  ),
  tf.keras.layers.Dense(
      units=3,
      activation=tf.keras.activations.softmax
  )
])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_43 (Conv2D)          (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_43 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_44 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 17, 17, 128)    

In [157]:
#gunakan loss categorical_crossentropy
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(
    optimizer=rmsprop_optimizer,
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

In [162]:
from sklearn.utils import validation
#train model
# Biar cantik aja progress bar nya
import sys
!{sys.executable} -m pip install tensorflow-addons
import tensorflow_addons as tfa
tqdm_callback = tfa.callbacks.TQDMProgressBar()

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    steps_per_epoch=25,
    validation_steps=5,
    callbacks=[tqdm_callback],
    verbose=0
)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Training:   0%|           0/15 ETA: ?s,  ?epochs/s

Epoch 1/15


0/25           ETA: ?s - 

Epoch 2/15


...

0/25           ETA: ?s - 

Epoch 14/15


0/25           ETA: ?s - 

Epoch 15/15


0/25           ETA: ?s - 

In [163]:
#save model
model.save('Model3.h5')